In [1]:
from typing import Tuple
import pandas as pd
from tqdm.notebook import tqdm
import sys
sys.path.append('..')
from evaluation.generated_dataset import GeneratedDataset, load_all_from_config
from evaluation.novelty import NoveltyFilter, filter_by_unique_structure

/home/kna/.cache/pypoetry/virtualenvs/wyckofftransformer-FeCwefly-py3.12/lib/python3.12/site-packages/matminer/utils/data.py:326: UserWarning: MagpieData(impute_nan=False):
In a future release, impute_nan will be set to True by default.
                    This means that features that are missing or are NaNs for elements
                    from the data source will be replaced by the average of that value
                    over the available elements.
                    This avoids NaNs after featurization that are often replaced by
                    dataset-dependent averages.
  warnings.warn(f"{self.__class__.__name__}(impute_nan=False):\n" + IMPUTE_NAN_WARNING)


In [2]:
raw_datasets = {
    "WyFormer": ("WyckoffTransformer", "CrySPR", "CHGNet_fix")
}

In [3]:
all_datasets = load_all_from_config(
    datasets=list(raw_datasets.values()) + \
        [("split", "train"), ("split", "val"), ("split", "test")],
    dataset_name="mpts_52")

In [4]:
novelty_reference = pd.concat([
    all_datasets[('split', 'train')].data,
    all_datasets[('split', 'val')].data], axis=0, verify_integrity=True)
novelty_filter = NoveltyFilter(novelty_reference)

In [5]:
import evaluation.statistical_evaluator
test_evaluator = evaluation.statistical_evaluator.StatisticalEvaluator(
    all_datasets[('split', 'test')].data, cdvae_eval_model_name="mp20")

In [6]:
mpts_52 = pd.concat([
    all_datasets[('split', 'train')].data,
    all_datasets[('split', 'val')].data,
    all_datasets[('split', 'test')].data], axis=0, verify_integrity=True)

In [7]:
train_evaluator = evaluation.statistical_evaluator.StatisticalEvaluator(
    all_datasets[('split', 'train')].data, cdvae_eval_model_name="mp20")

In [18]:
train_evaluator.test_dataset.smact_validity.mean()

0.8710372534696859

In [21]:
train_evaluator.test_dataset.structural_validity.mean()

1.0

In [29]:
frozenset(range(1000)) - frozenset(all_datasets[("WyckoffTransformer", "CrySPR", "CHGNet_fix")].data.index)

frozenset({278, 399, 467, 535, 697})

In [24]:
(train_evaluator.test_dataset.spacegroup_number == 1).sum() / len(train_evaluator.test_dataset)

0.0023009495982468956

In [8]:
train_w_template_set = frozenset(novelty_reference.apply(evaluation.novelty.record_to_anonymous_fingerprint, axis=1))

In [9]:
train_evaluator.get_num_sites_emd(all_datasets[('split', 'test')].data)

2.141915765380806

In [10]:
mpts_52.fingerprint.nunique(), mpts_52.shape[0]

(39907, 40476)

In [11]:
mpts_52.fingerprint.nunique() / mpts_52.shape[0]

0.985942286787232

In [12]:
mpts_52.smact_validity.mean()

0.8824488585828639

In [25]:
table = pd.DataFrame(
    index=raw_datasets.keys(), columns=[
        "Novelty (%)", "Structural", "Compositional", 
        "Recall", "Precision",
        r"$\rho$", "$E$", "# Elements",
        "Novel Template (%)", 
        "Novel AFLOW on P1",
        "Novel AFLOW on !P1",
        "Novel AFLOW (%)",
        "Novel AFLOW relaxed (%)",
        "P1 (%)",
        "Space Group"])
table.index.name = "Method"
E_hull_threshold = 0.08
for name, transformations in tqdm(raw_datasets.items()):
    dataset = all_datasets[transformations]
    unique = filter_by_unique_structure(dataset.data)
    print(f"Unique: {len(unique)} / {len(dataset.data)} = {len(unique) / len(dataset.data)}")
    novel_template = ~unique.apply(evaluation.novelty.record_to_anonymous_fingerprint, axis=1).isin(train_w_template_set)
    table.loc[name, "Novel Template (%)"] = 100 * novel_template.mean()
    if transformations == ("split", "train"):
        novel = unique
    else:
        novel = novelty_filter.get_novel(unique)
    table.loc[name, "Novelty (%)"] = 100 * len(novel) / len(unique)
    if "structural_validity" in novel.columns:
        table.loc[name, "Structural"] = 100 * novel.structural_validity.mean()
        table.loc[name, "Compositional"] = 100 * novel.smact_validity.mean()
    if "cdvae_crystal" in novel.columns:
        #cov_metrics = test_evaluator.get_coverage(novel.cdvae_crystal)    
        #table.loc[name, "Recall"] = 100 * cov_metrics["cov_recall"]
        #table.loc[name, "Precision"] = 100 * cov_metrics["cov_precision"]
        novel = novel[novel.structural_validity]
        all_templates = novel.apply(evaluation.novelty.record_to_anonymous_fingerprint, axis=1)
        novel_template = ~all_templates.isin(train_w_template_set)
        table.loc[name, "Novel Template (%)"] = 100 * novel_template.mean()
        table.loc[name, "Novel Uniques Templates (#)"] = all_templates[novel_template].nunique() 
        table.loc[name, r"$\rho$"] = train_evaluator.get_density_emd(novel)
        table.loc[name, "$E$"] = train_evaluator.get_cdvae_e_emd(novel)
        table.loc[name, "# Elements"] = train_evaluator.get_num_elements_emd(novel)
    table.loc[name, "P1 (%)"] = 100 * (novel.group == 1).mean()
    table.loc[name, "Space Group"] = train_evaluator.get_sg_chi2(novel)
table

  0%|          | 0/1 [00:00<?, ?it/s]

Unique: 995 / 995 = 1.0


,Novelty (%),Structural,Compositional,Recall,Precision,$\rho$,$E$,# Elements,Novel Template (%),Novel AFLOW on P1,Novel AFLOW on !P1,Novel AFLOW (%),Novel AFLOW relaxed (%),P1 (%),Space Group,Novel Uniques Templates (#)
Method,,,,,,,,,,,,,,,,
WyFormer,98.693467,99.287169,76.680244,NaN,NaN,0.697912,0.108448,0.228226,44.512821,NaN,NaN,NaN,NaN,0.0,0.225362,386.0


In [12]:
cdvae_table = pd.DataFrame(index=pd.Index(raw_datasets.keys(), tupleize_cols=False),
    columns=[
        "Structural", "Compositional",
        "Recall", "Precision",
        r"$\rho$", "$E$", "# Elements"])
sample_size = 900
for name, transformations in tqdm(raw_datasets.items()):
    dataset = all_datasets[transformations]
    if "structure" in dataset.data.columns:
        cdvae_table.loc[name, "Compositional"] = 100*dataset.data.smact_validity.mean()
        cdvae_table.loc[name, "Structural"] = 100*dataset.data.structural_validity.mean()
        valid = dataset.data[dataset.data.naive_validity]
        cov_metrics = test_evaluator.get_coverage(valid.cdvae_crystal, sample_size)
        cdvae_table.loc[name, "Recall"] = 100*cov_metrics["cov_recall"]
        cdvae_table.loc[name, "Precision"] = 100*cov_metrics["cov_precision"]
        cdvae_table.loc[name, r"$\rho$"] = test_evaluator.get_density_emd(valid)
        cdvae_table.loc[name, "$E$"] = test_evaluator.get_cdvae_e_emd(valid)
        cdvae_table.loc[name, "# Elements"] = test_evaluator.get_num_elements_emd(valid)
cdvae_table.to_csv("tables/cdvae_metrics_no_relax_table.csv")
#prettify(cdvae_table).to_latex("tables/cdvae_metrics_no_relax_table.tex", siunitx=True, convert_css=True)
#prettify(cdvae_table)

  0%|          | 0/1 [00:00<?, ?it/s]

In [13]:
cdvae_table

,Structural,Compositional,Recall,Precision,$\rho$,$E$,# Elements
WyFormer,NaN,NaN,NaN,NaN,NaN,NaN,NaN
